# perparing the data

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn
import seaborn as sbe
import os 
import math 
import shutil
from glob import glob
from numpy import array
from imutils import paths
from tensorflow.keras import models,layers
import cv2

In [3]:
training_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Training\\*\\*",shuffle=False)
testing_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Testing\\*\\*",shuffle=False)
validation_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Validation\\*\\*",shuffle=False)
for f in training_ds.take(4):
    print(f.numpy())
for f in testing_ds.take(4):
    print(f.numpy())
for f in validation_ds.take(4):
    print(f.numpy())


b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_1462.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_4424.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_5215.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_5539.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_100.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_1635.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_1711.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_4911.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_100_0_982.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_101_0_2726.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_101_0_5413.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_102_0_5905.jpg'


In [4]:
class_name=["CaS","CoS","Gum","MC","OC","OLP","OT"]
training_ds = training_ds.shuffle(200)
testing_ds = testing_ds.shuffle(200)
validation_ds = validation_ds.shuffle(200)
image_train_count=len(training_ds)
print(image_train_count)
print(len(testing_ds))
print(len(validation_ds))


3087
1028
1028


In [5]:
#s="Teeth DataSet\\Teeth_Dataset\\Training\\MC"
#s.split("\\")[-1]
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]


In [6]:
training_ds.map(get_label)

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [7]:
def process_image(file_path):
    label=get_label(file_path)
    img = tf.io.read_file(file_path)  # Read the image file
    img = tf.image.decode_jpeg(img, channels=3)  # Decode as JPEG
    img = tf.image.resize(img, [256, 256])  # Resize to 256x256
    return img, label  # Return image and label as a tuple


#training_ds = training_ds.map(process_image)

In [8]:
img, label = process_image("Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_1462.jpg")
img.numpy()[:2]

array([[[252., 217., 197.],
        [252., 217., 197.],
        [252., 217., 197.],
        ...,
        [245., 244., 249.],
        [245., 244., 249.],
        [245., 244., 249.]],

       [[252., 217., 197.],
        [252., 217., 197.],
        [252., 217., 197.],
        ...,
        [245., 244., 249.],
        [245., 244., 249.],
        [245., 244., 249.]]], dtype=float32)

In [9]:
training_ds=training_ds.map(process_image)
testing_ds=testing_ds.map(process_image)
validation_ds=validation_ds.map(process_image)
for img,label in training_ds.take(2):
    print(img)
    

tf.Tensor(
[[[ 12.  16.  15.]
  [ 12.  16.  15.]
  [ 12.  16.  15.]
  ...
  [255. 211. 182.]
  [253. 208. 179.]
  [254. 209. 180.]]

 [[ 12.  16.  15.]
  [ 12.  16.  15.]
  [ 12.  16.  15.]
  ...
  [255. 212. 183.]
  [255. 211. 182.]
  [255. 213. 184.]]

 [[ 12.  16.  15.]
  [ 12.  16.  15.]
  [ 12.  16.  15.]
  ...
  [254. 213. 185.]
  [255. 213. 186.]
  [255. 216. 188.]]

 ...

 [[232. 158. 113.]
  [232. 158. 113.]
  [232. 158. 113.]
  ...
  [231. 134. 128.]
  [231. 134. 128.]
  [233. 136. 130.]]

 [[232. 158. 113.]
  [232. 158. 113.]
  [232. 158. 113.]
  ...
  [231. 134. 128.]
  [231. 134. 128.]
  [233. 136. 130.]]

 [[232. 158. 113.]
  [232. 158. 113.]
  [232. 158. 113.]
  ...
  [231. 134. 128.]
  [231. 134. 128.]
  [233. 136. 130.]]], shape=(256, 256, 3), dtype=float32)
tf.Tensor(
[[[218. 164. 136.]
  [217. 163. 135.]
  [216. 162. 134.]
  ...
  [251. 192. 176.]
  [251. 189. 174.]
  [249. 187. 172.]]

 [[216. 162. 134.]
  [216. 162. 134.]
  [216. 162. 134.]
  ...
  [247. 186. 168.]

In [10]:
for image, label in training_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[251. 191. 193.]
  [251. 191. 193.]
  [251. 191. 193.]
  ...
  [ 64.  83. 100.]
  [ 64.  83. 100.]
  [ 64.  83. 100.]]

 [[252. 192. 194.]
  [252. 192. 194.]
  [252. 192. 194.]
  ...
  [ 64.  83. 100.]
  [ 64.  83. 100.]
  [ 64.  83. 100.]]

 [[253. 193. 195.]
  [253. 193. 195.]
  [253. 193. 195.]
  ...
  [ 64.  83. 100.]
  [ 64.  83. 100.]
  [ 64.  83. 100.]]

 ...

 [[245. 181. 179.]
  [245. 181. 179.]
  [245. 181. 179.]
  ...
  [ 96. 115. 132.]
  [ 96. 115. 132.]
  [ 96. 115. 132.]]

 [[245. 181. 179.]
  [245. 181. 179.]
  [245. 181. 179.]
  ...
  [ 96. 115. 132.]
  [ 96. 115. 132.]
  [ 96. 115. 132.]]

 [[246. 182. 180.]
  [246. 182. 180.]
  [246. 182. 180.]
  ...
  [ 95. 114. 131.]
  [ 95. 114. 131.]
  [ 95. 114. 131.]]], shape=(256, 256, 3), dtype=float32)
**** tf.Tensor(b'CaS', shape=(), dtype=string)


In [11]:
def scale(image, label):
    return image/255, label

In [12]:
training_ds=training_ds.map(scale)
testing_ds=testing_ds.map(scale)
validation_ds=validation_ds.map(scale)

In [13]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_train = []
y_train = []

for img, label in training_ds.as_numpy_iterator():
    x_train.append(img)
    y_train.append(label)

# Convert lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape, y_train.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(3087, 256, 256, 3) (3087,)


In [14]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_test = []
y_test = []

for img, label in testing_ds.as_numpy_iterator():
    x_test.append(img)
    y_test.append(label)

# Convert lists to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape, y_test.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(1028, 256, 256, 3) (1028,)


In [15]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_val = []
y_val = []

for img, label in validation_ds.as_numpy_iterator():
    x_val.append(img)
    y_val.append(label)

# Convert lists to numpy arrays
x_val = np.array(x_val)
y_val = np.array(y_val)

print(x_val.shape, y_val.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(1028, 256, 256, 3) (1028,)


In [16]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')

In [17]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder and transform string labels to integer labels
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
y_val = label_encoder.fit_transform(y_val)
print(y_train)

[0 0 0 ... 6 6 6]


In [18]:
print(x_train.shape)
print(y_train.shape)
y_train = y_train.reshape(-1,)
print([y_train])

(3087, 256, 256, 3)
(3087,)
[array([0, 0, 0, ..., 6, 6, 6], dtype=int64)]


here we prepare the data to be used in the modeling process

# Preparing the model

In [19]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax')
])

c:\Users\shahd\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [21]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,935 (60.14 MB)

 Trainable params: 15,764,935 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
cnn.fit(x_train, y_train, epochs=10)

Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 397ms/step - accuracy: 0.1922 - loss: 3.2093
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 39s 402ms/step - accuracy: 0.4058 - loss: 1.5524
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 53s 549ms/step - accuracy: 0.6101 - loss: 1.0981
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 41s 424ms/step - accuracy: 0.7848 - loss: 0.6535
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 52s 535ms/step - accuracy: 0.9108 - loss: 0.3021
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 42s 429ms/step - accuracy: 0.9712 - loss: 0.1320
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 44s 452ms/step - accuracy: 0.9931 - loss: 0.0516
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 63s 647ms/step - accuracy: 0.9956 - loss: 0.0209
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 42s 432ms/step - accuracy: 1.0000 - loss: 0.0055
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 364ms/step - accuracy: 1.0000 - loss: 0.0021


In [26]:
model_eval=cnn.evaluate(x_test,y_test)


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - accuracy: 0.5849 - loss: 2.1540
